## 01.- Red neuronal artificial

### a) Importación de las librerías

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.14.0'

## Parte 1 - Preprocesamiento de datos

### Importar el conjunto de datos

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Codificación de datos categóricos

Etiqueta Codificación de la columna "Género"

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Codificación de la columna "Geografía"

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### División del conjunto de datos en conjunto de entrenamiento y conjunto de prueba

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de características

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Parte 2 - Construcción de la RNA

### Inicialización de la RNA

In [12]:
ann = tf.keras.models.Sequential()

### Añadir la capa de entrada y la primera capa oculta

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Añadir la segunda capa oculta

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Añadir la capa de salida

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Parte 3 - Entrenamiento de la RNA

### Compilación de la RNA

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Entrenamiento de la RNA en el conjunto de entrenamiento

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.6396 - accuracy: 0.6744
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4809 - accuracy: 0.8010
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4466 - accuracy: 0.8055
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4361 - accuracy: 0.8062
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4307 - accuracy: 0.8065
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4263 - accuracy: 0.8079
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4205 - accuracy: 0.8116
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4121 - accuracy: 0.8176
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4030 - accuracy: 0.8270
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3944 - accura

## Parte 4 - Realización de predicciones y evaluación del modelo

### Predecir el resultado de una sola observación

**Tarea**

Utilice nuestro modelo RNA para predecir si el cliente con la siguiente información abandonará el banco:

Geografía: Francia

Puntuación crediticia: 600

Sexo: Hombre

Edad: 40 años

Tenencia: 3 años

Saldo: \$ 60,000

Número de productos: 2

¿Este cliente tiene tarjeta de crédito? Sí

Es este cliente un Miembro Activo: Sí

Salario estimado: \$ 50000

Entonces, ¿debemos despedirnos de ese cliente?

**Solución**

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 231ms/step
[[False]]


Por tanto, nuestro modelo RNA predice que este cliente se queda en el banco.

**Nota importante 1:** Observe que los valores de las características se introdujeron todos en un par doble de corchetes. Esto se debe a que el método "predict" siempre espera un array 2D como formato de entrada. Y poner nuestros valores en un par doble de corchetes hace que la entrada sea exactamente un array 2D.

**Nota importante 2:** Observe también que el país "Francia" no se introdujo como cadena en la última columna, sino como "1, 0, 0" en las tres primeras. Eso es porque, por supuesto, el método de predicción espera los valores codificados de un solo golpe del estado, y como vemos en la primera fila de la matriz de características X, "Francia" se codificó como "1, 0, 0". Y cuidado con incluir estos valores en las tres primeras columnas, porque las variables ficticias siempre se crean en las primeras columnas.

### Predicción de los resultados de la prueba

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 3ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Hacer la Matríz de Confusión

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1529   66]
 [ 200  205]]


0.867

Una Red Neuronal Artificial (RNA) es un modelo de aprendizaje automático inspirado en la estructura y funcionamiento del cerebro humano. Consiste en una red de unidades llamadas neuronas artificiales que trabajan en conjunto para realizar tareas de procesamiento de información, como clasificación, regresión y reconocimiento de patrones. A continuación, proporcionaré una descripción detallada de las redes neuronales artificiales con enfoque matemático:

### 1. Neurona Artificial
Una neurona artificial es la unidad fundamental de una RNA. Cada neurona toma una o varias entradas, las procesa y produce una salida. Matemáticamente, la salida de una neurona se calcula como una función de combinación lineal de las entradas, seguida de una función de activación. La ecuación general de una neurona es:

$$ y = f(\sum_{i=1}^{n} w_i x_i + b) $$

Donde:
- $y$ es la salida de la neurona.
- $f$ es la función de activación.
- $x_i$ son las entradas.
- $w_i$ son los pesos asociados a las entradas.
- $b$ es el sesgo (bias).

### 2. Capas de Neuronas
Una RNA generalmente se organiza en capas de neuronas. Las capas se dividen en tres tipos principales:

#### a. Capa de Entrada
Esta capa recibe las características de entrada y transmite la información a las capas ocultas o de salida. No realiza ningún cálculo adicional.

#### b. Capas Ocultas
Estas capas procesan la información mediante la aplicación de transformaciones no lineales a las entradas. Pueden ser múltiples y se utilizan para aprender representaciones intermedias de los datos.

#### c. Capa de Salida
Esta capa produce la salida final de la red, que depende de la tarea específica. Por ejemplo, en un problema de clasificación, la capa de salida puede utilizar una función de activación softmax para producir probabilidades de pertenencia a diferentes clases.

### 3. Funciones de Activación
Las funciones de activación introducen no linealidad en la RNA y son esenciales para que la red pueda aprender relaciones complejas en los datos. Algunas funciones de activación comunes son:

- **Sigmoide (Logistic):** $f(z) = \frac{1}{1 + e^{-z}}$
- **ReLU (Rectified Linear Unit):** $f(z) = \max(0, z)$
- **Tanh (Tangente hiperbólica):** $f(z) = \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}}$

### 4. Propagación hacia Adelante (Forward Propagation)
La propagación hacia adelante es el proceso de calcular la salida de la red neuronal desde la capa de entrada hasta la capa de salida. Se realiza aplicando la ecuación de una neurona a cada neurona en cada capa, comenzando desde la capa de entrada y avanzando hacia la capa de salida.

### 5. Entrenamiento de la RNA
El entrenamiento de una RNA implica ajustar los pesos y sesgos de las neuronas para que la red pueda realizar tareas específicas. Esto se logra a través de un proceso llamado retropropagación del error (Backpropagation). El objetivo es minimizar una función de costo que mide la discrepancia entre las predicciones de la red y los valores reales.

### 6. Función de Costo
La función de costo $J$ cuantifica la diferencia entre las predicciones de la RNA y los valores reales. La función de costo depende de la tarea específica y puede ser, por ejemplo, el error cuadrático medio (MSE) para regresión o la entropía cruzada para clasificación.

### 7. Optimización
La optimización se utiliza para ajustar los pesos y sesgos de las neuronas con el fin de minimizar la función de costo. Algunos algoritmos de optimización comunes son el descenso de gradiente (Gradient Descent) y sus variantes, como el descenso de gradiente estocástico (SGD) y el descenso de gradiente con momento (Momentum).

### 8. Overfitting y Regularización
El overfitting es un problema común en RNA, donde la red se ajusta demasiado a los datos de entrenamiento y no generaliza bien a nuevos datos. Para abordar este problema, se utilizan técnicas de regularización, como la regularización L1 y L2, que penalizan los pesos grandes.

### 9. Arquitecturas de Redes Neuronales
Existen muchas arquitecturas de RNA, como redes neuronales feedforward, redes neuronales recurrentes (RNN), redes neuronales convolucionales (CNN) y más. Cada arquitectura tiene aplicaciones específicas en tareas de aprendizaje automático.

En resumen, una Red Neuronal Artificial es un modelo de aprendizaje automático que se basa en la estructura y funcionamiento de las neuronas biológicas. Se compone de capas de neuronas, utiliza funciones de activación para introducir no linealidad y se entrena mediante retropropagación del error. Las RNA son ampliamente utilizadas en una variedad de aplicaciones de aprendizaje automático y han demostrado ser efectivas para resolver problemas complejos.